In [1]:
### Merge population and employment tensor to one input tensor
### population_tensor.h5 strata:
### 'Male 0-15', 'Male 16-24', 'Male 25-49', 'Male 50-64', 'Male 65+', 'Female 0-15', 'Female 16-24', 'Female 25-49', 'Female 50-64', 'Female 65+'
### employment_tensor.h5 strata:
### full-time-sector1-18, part-time-sector1-18, total employment-sector1-18
### sector code 1-18:
# 1 : Agriculture, forestry & fishing (A)
# 2 : Mining, quarrying & utilities (B,D and E)
# 3 : Manufacturing (C)
# 4 : Construction (F)
# 5 : Motor trades (Part G)
# 6 : Wholesale (Part G)
# 7 : Retail (Part G)
# 8 : Transport & storage (inc postal) (H)
# 9 : Accommodation & food services (I)
# 10 : Information & communication (J)
# 11 : Financial & insurance (K)
# 12 : Property (L)
# 13 : Professional, scientific & technical (M)
# 14 : Business administration & support services (N)
# 15 : Public administration & defence (O)
# 16 : Education (P)
# 17 : Health (Q)
# 18 : Arts, entertainment, recreation & other services (R,S,T and U)

In [13]:
import h5py
import numpy as np

# Paths
POP_FILE = 'population/population_tensor.h5'
EMP_FILE = 'employment/employment_tensor.h5'

# Define labels for the years and strata
YEARS = list(range(2015, 2023))  # 8 years

# Read the HDF5 file
print("Reading tensor from HDF5 file...")
with h5py.File(POP_FILE, 'r') as h5f:
    population_tensor = h5f['population_tensor'][:]
    print(f"Loaded tensor shape: {population_tensor.shape}")
    print(f"Loaded tensor dtype: {population_tensor.dtype}")

NUM_YEARS, NUM_ROWS, NUM_COLS, POP_NUM_STRATA = population_tensor.shape

# Load the employment tensor
print("Loading the employment tensor...")
with h5py.File(EMP_FILE, 'r') as h5f:
    employment_tensor = h5f['employment_tensor'][:]
    print(f"Loaded tensor shape: {employment_tensor.shape}")
    print(f"Loaded tensor dtype: {employment_tensor.dtype}")

_, _, _, EMP_NUM_STRATA = employment_tensor.shape


Reading tensor from HDF5 file...
Loaded tensor shape: (8, 653, 573, 10)
Loaded tensor dtype: float32
Loading the employment tensor...
Loaded tensor shape: (8, 653, 573, 54)
Loaded tensor dtype: float32


In [14]:
# Initialize final tensor
new_channels = 94  # Base channels + expanded employment strata
final_tensor = np.zeros((NUM_YEARS, NUM_ROWS, NUM_COLS, new_channels))

# Population aggregations
final_tensor[..., 0] = np.sum(population_tensor, axis=-1)  # Total population
final_tensor[..., 1] = np.sum(population_tensor[..., :5], axis=-1)  # Total male
final_tensor[..., 2] = np.sum(population_tensor[..., 5:], axis=-1)  # Total female

for i in range(5):
    final_tensor[..., 3 + i] = population_tensor[..., i] + population_tensor[..., i + 5]  # Both sexes by age strata

# Copy original population strata
final_tensor[..., 8:18] = population_tensor

# Employment aggregations
final_tensor[..., 18] = np.sum(employment_tensor[..., 36:54], axis=-1)  # Total employment
final_tensor[..., 19] = np.sum(employment_tensor[..., :18], axis=-1)  # Full-time employment
final_tensor[..., 20] = np.sum(employment_tensor[..., 18:36], axis=-1)  # Part-time employment
final_tensor[..., 21] = final_tensor[..., 18] - (final_tensor[..., 19] + final_tensor[..., 20])  # Self-employed

for i in range(18):
    final_tensor[..., 22 + i] = employment_tensor[..., 36 + i]  # Total employment by sector

# Copy employment strata by sector and type
offset = 40
final_tensor[..., offset:offset + 18] = employment_tensor[..., :18]  # Full-time
final_tensor[..., offset + 18:offset + 36] = employment_tensor[..., 18:36]  # Part-time
final_tensor[..., offset + 36:offset + 54] = employment_tensor[..., 36:54] - (
    employment_tensor[..., :18] + employment_tensor[..., 18:36]
)  # Self-employed

In [15]:
import numpy as np

# Calculate the sum for each channel in the final tensor
channel_sums = np.sum(final_tensor, axis=(0, 1, 2))

# Print the sum for each channel
for i, channel_sum in enumerate(channel_sums):
    print(f"Channel {i}: Sum = {channel_sum}")

Channel 0: Sum = 472610859.64330274
Channel 1: Sum = 231832885.75688055
Channel 2: Sum = 240777973.789977
Channel 3: Sum = 88350465.48638155
Channel 4: Sum = 51322029.181850545
Channel 5: Sum = 156804762.87330672
Channel 6: Sum = 89769651.86025704
Channel 7: Sum = 86363950.19764414
Channel 8: Sum = 45281846.19835183
Channel 9: Sum = 25989700.581955124
Channel 10: Sum = 76924922.45417456
Channel 11: Sum = 44201339.90919852
Channel 12: Sum = 39435076.63078077
Channel 13: Sum = 43068619.29719775
Channel 14: Sum = 25332328.604532067
Channel 15: Sum = 79879840.40506504
Channel 16: Sum = 45568311.953469776
Channel 17: Sum = 46928873.558766276
Channel 18: Sum = 221850088.83767548
Channel 19: Sum = 147050831.60725993
Channel 20: Sum = 67915998.02473487
Channel 21: Sum = 6883259.205680726
Channel 22: Sum = 305479.9408880193
Channel 23: Sum = 2566374.9462763974
Channel 24: Sum = 17599474.872870687
Channel 25: Sum = 10843259.804212632
Channel 26: Sum = 4054369.999852711
Channel 27: Sum = 8619209.

In [16]:
# Save the final tensor to HDF5
with h5py.File("population_and_employment.h5", 'w') as h5f:
    h5f.create_dataset('features', data=final_tensor)

In [18]:
channel_descriptions = [
    "Total population (sum of all population strata)", #0
    "Total male population (sum of population strata for males: 0-15, 16-24, 25-49, 50-64, 65+)", #1
    "Total female population (sum of population strata for females: 0-15, 16-24, 25-49, 50-64, 65+)", #2
    "Total population (both sexes) aged 0-15", #3
    "Total population (both sexes) aged 16-24", #4
    "Total population (both sexes) aged 25-49", #5
    "Total population (both sexes) aged 50-64", #6
    "Total population (both sexes) aged 65+", #7
    "Male population aged 0-15", #8
    "Male population aged 16-24", #9
    "Male population aged 25-49", #10
    "Male population aged 50-64", #11
    "Male population aged 65+", #12
    "Female population aged 0-15", #13
    "Female population aged 16-24", #14
    "Female population aged 25-49", #15
    "Female population aged 50-64", #16
    "Female population aged 65+", #17
    "Total employment (sum of all employment types: full-time, part-time, self-employed)", #18
    "Total full-time employment (sum across all sectors)", #19
    "Total part-time employment (sum across all sectors)", #20
    "Total self-employed (calculated as total employment - (full-time + part-time))", #21
    "Total employment by sector (sector 1: Agriculture, forestry & fishing)", #22
    "Total employment by sector (sector 2: Mining, quarrying & utilities)", #23
    "Total employment by sector (sector 3: Manufacturing)", #24
    "Total employment by sector (sector 4: Construction)", #25
    "Total employment by sector (sector 5: Motor trades)", #26
    "Total employment by sector (sector 6: Wholesale)", #27
    "Total employment by sector (sector 7: Retail)", #28
    "Total employment by sector (sector 8: Transport & storage)", #29
    "Total employment by sector (sector 9: Accommodation & food services)", #30
    "Total employment by sector (sector 10: Information & communication)", #31
    "Total employment by sector (sector 11: Financial & insurance)", #32
    "Total employment by sector (sector 12: Property)", #33
    "Total employment by sector (sector 13: Professional, scientific & technical)", #34
    "Total employment by sector (sector 14: Business administration & support services)", #35
    "Total employment by sector (sector 15: Public administration & defence)", #36
    "Total employment by sector (sector 16: Education)", #37
    "Total employment by sector (sector 17: Health)", #38
    "Total employment by sector (sector 18: Arts, entertainment, recreation & other services)", #39
    "Full-time employment by sector (sector 1: Agriculture, forestry & fishing)", #40
    "Full-time employment by sector (sector 2: Mining, quarrying & utilities)", #41
    "Full-time employment by sector (sector 3: Manufacturing)", #42
    "Full-time employment by sector (sector 4: Construction)", #43
    "Full-time employment by sector (sector 5: Motor trades)", #44
    "Full-time employment by sector (sector 6: Wholesale)", #45
    "Full-time employment by sector (sector 7: Retail)", #46
    "Full-time employment by sector (sector 8: Transport & storage)", #47
    "Full-time employment by sector (sector 9: Accommodation & food services)", #48
    "Full-time employment by sector (sector 10: Information & communication)", #49
    "Full-time employment by sector (sector 11: Financial & insurance)", #50
    "Full-time employment by sector (sector 12: Property)", #51
    "Full-time employment by sector (sector 13: Professional, scientific & technical)", #52
    "Full-time employment by sector (sector 14: Business administration & support services)", #53
    "Full-time employment by sector (sector 15: Public administration & defence)", #54
    "Full-time employment by sector (sector 16: Education)", #55
    "Full-time employment by sector (sector 17: Health)", #56
    "Full-time employment by sector (sector 18: Arts, entertainment, recreation & other services)", #57
    "Part-time employment by sector (sector 1: Agriculture, forestry & fishing)", #58
    "Part-time employment by sector (sector 2: Mining, quarrying & utilities)", #59
    "Part-time employment by sector (sector 3: Manufacturing)", #60
    "Part-time employment by sector (sector 4: Construction)", #61
    "Part-time employment by sector (sector 5: Motor trades)", #62
    "Part-time employment by sector (sector 6: Wholesale)", #63
    "Part-time employment by sector (sector 7: Retail)", #64
    "Part-time employment by sector (sector 8: Transport & storage)", #65
    "Part-time employment by sector (sector 9: Accommodation & food services)", #66
    "Part-time employment by sector (sector 10: Information & communication)", #67
    "Part-time employment by sector (sector 11: Financial & insurance)", #68
    "Part-time employment by sector (sector 12: Property)", #69
    "Part-time employment by sector (sector 13: Professional, scientific & technical)", #70
    "Part-time employment by sector (sector 14: Business administration & support services)", #71
    "Part-time employment by sector (sector 15: Public administration & defence)", #72
    "Part-time employment by sector (sector 16: Education)", #73
    "Part-time employment by sector (sector 17: Health)", #74
    "Part-time employment by sector (sector 18: Arts, entertainment, recreation & other services)", #75
    "Self-employed by sector (sector 1: Agriculture, forestry & fishing)", #76
    "Self-employed by sector (sector 2: Mining, quarrying & utilities)", #77
    "Self-employed by sector (sector 3: Manufacturing)", #78
    "Self-employed by sector (sector 4: Construction)", #79
    "Self-employed by sector (sector 5: Motor trades)", #80
    "Self-employed by sector (sector 6: Wholesale)", #81
    "Self-employed by sector (sector 7: Retail)", #82
    "Self-employed by sector (sector 8: Transport & storage)", #83
    "Self-employed by sector (sector 9: Accommodation & food services)", #84
    "Self-employed by sector (sector 10: Information & communication)", #85
    "Self-employed by sector (sector 11: Financial & insurance)", #86
    "Self-employed by sector (sector 12: Property)", #87
    "Self-employed by sector (sector 13: Professional, scientific & technical)", #88
    "Self-employed by sector (sector 14: Business administration & support services)", #89
    "Self-employed by sector (sector 15: Public administration & defence)", #90
    "Self-employed by sector (sector 16: Education)", #91
    "Self-employed by sector (sector 17: Health)", #92
    "Self-employed by sector (sector 18: Arts, entertainment, recreation & other services)" #93
]

In [19]:
len(channel_descriptions)

94

In [3]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

# Load the tensor from the HDF5 file
file_path = 'population_and_employment.h5'
output_image_path = 'population_employment_log10.png'

with h5py.File(file_path, 'r') as f:
    data = f['features'][:]  # Assuming the dataset inside is named 'tensor'

# Average across the years (axis 0)
average_data = np.mean(data, axis=0)  # Shape: [rows, cols, channels]

# Extract the 0th and 18th channels and apply log10 scaling
population = np.log10(average_data[:, :, 0] + 1)  # Total population (log10 scale)
employment = np.log10(average_data[:, :, 18] + 1)  # Total employment (log10 scale)

# Plot the heat maps
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Total population heat map
im0 = axes[0].imshow(population, cmap='viridis', interpolation='nearest', aspect="auto")
axes[0].set_title("Log10 Average Total Population")
axes[0].axis('off')
fig.colorbar(im0, ax=axes[0])

# Total employment heat map
im1 = axes[1].imshow(employment, cmap='viridis', interpolation='nearest', aspect="auto")
axes[1].set_title("Log10 Average Total Employment")
axes[1].axis('off')
fig.colorbar(im1, ax=axes[1])

# Save the image to file
plt.tight_layout()
plt.savefig(output_image_path)
plt.close()